# Fix ses 02 for sub 02

Problem: the phase encoding direction for the functionals was erroneously set to left-right instead of head-foot. In addition, the phase encoding direction for the top-ups was set to the same direction, so unwarping was not possible in this dataset. 

To solve this problem, we can FNIRT all the functionals from session 2 to one of the functionals from session 3. Then, we put the fmap from that run as the fmap for all functionals in session 2. 

In [8]:
import os
import nibabel as nb
import numpy as np
from nipype.interfaces import fsl
import glob
base_dir = '/home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/'
target_dir = '/home/raw_data/2018/visual/cerebellum_prf/bids_ses03/sub-02/ses-03/func/'

Let's first create a time-average from an example run in session 03

In [7]:

target_fn = os.path.join(target_dir,'sub-02_ses-03_task-prf_run-01_bold.nii.gz')
target_avg_fn = os.path.join(base_dir,'target.nii.gz')

img = nb.load(target_fn)
data = img.get_data()
avg_data = np.mean(data,axis=-1)
new_img = nb.Nifti1Image(avg_data,img.affine,img.header)
nb.save(new_img,target_avg_fn)
    

now that we have the target, let's first flirt, and then fnirt the ses 2 functionals to the target epi. For that, let's first create time averages to save computation time

In [48]:
# first create time averages to save computation time of flirt and fnirts
for fn in glob.glob(os.path.join(base_dir,'*bold.nii.gz')):
    
    img = nb.load(fn)
    data = img.get_data()
    avg_data = np.mean(data,axis=-1)
    new_img = nb.Nifti1Image(avg_data,img.affine,img.header)
    out_fn = fn.replace('.nii.gz','_avg.nii.gz')
    nb.save(new_img,out_fn)


In [10]:
# first flirt all files to target 
for fn in glob.glob(os.path.join(base_dir,'*bold_avg.nii.gz')):
    
    print('now flirting %s to %s'%(fn,target_avg_fn))
    # now flirt avg fmap to target avg fmap
    mat_fn = fn.replace('.nii.gz','.mat')

    flt = fsl.FLIRT()
    flt.inputs.in_file = fn
    flt.inputs.reference = target_avg_fn
    flt.inputs.out_matrix_file = mat_fn
    flt.inputs.out_file = fn.replace('.nii.gz','_flirted.nii.gz')
    flt.run()

now flirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-01_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now flirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-02_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now flirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-08_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now flirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-05_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz


In [11]:
# now compute the warpfield of all time-average files to the time-average target
for fn in glob.glob(os.path.join(base_dir,'*bold_avg.nii.gz')):
    
    print('now fnirting %s to %s'%(fn,target_avg_fn))

    coef_fn = fn.replace('.nii.gz','_field.nii.gz')

    flt = fsl.FNIRT()
    flt.inputs.in_file = fn
    flt.inputs.affine_file = fn.replace('.nii.gz','.mat')
    flt.inputs.ref_file = target_avg_fn
    flt.inputs.warped_file = fn.replace('.nii.gz','_fnirted.nii.gz')
    flt.inputs.field_file = coef_fn
    flt.run()

now fnirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-01_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now fnirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-02_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now fnirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-08_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz
now fnirting /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-05_bold_avg.nii.gz to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/target.nii.gz


In [16]:
# and applywarp to full imgs
for fn in glob.glob(os.path.join(base_dir,'*bold.nii.gz')):
    
    print('now applying warp to %s'%(fn))
    coef_fn = fn.replace('.nii.gz','_avg_field.nii.gz')
    
    flt = fsl.ApplyWarp()
    flt.inputs.ref_file = target_avg_fn
    flt.inputs.in_file = fn
    flt.inputs.out_file = fn.replace('.nii.gz','_fnirted.nii.gz')
    flt.inputs.field_file = coef_fn
    flt.run()




now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-04_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-05_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-03_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-06_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-08_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-02/func/sub-02_ses-02_task-prf_run-01_bold.nii.gz
now applying warp to /home/raw_data/2018/visual/cerebellum_prf/bids_sub-02_former_ses-02/sub-02/ses-

In [ ]:
# next stuff is wrong old stuff

In [33]:
# load in target file and compute average over volumes
for fmap_fn in glob.glob(os.path.join(raw_dir,'fmap','*epi.nii.gz')):
    img = nb.load(fmap_fn)
    data = img.get_data()
    avg_data = np.mean(data,axis=-1)
    new_img = nb.Nifti1Image(avg_data,img.affine,img.header)
    out_fn = fmap_fn.replace('.nii.gz','_avg.nii.gz')
    nb.save(new_img,out_fn)

In [32]:
# now create a target img
for fmap_fn in glob.glob(os.path.join(raw_target_dir,'fmap','*epi.nii.gz')):
#     fmap_fn = os.path.join(raw_target_dir,'fmap','sub-02_ses-02_run-01_epi.nii.gz')
    img = nb.load(fmap_fn)
    data = img.get_data()
    avg_data = np.mean(data,axis=-1)
    new_img = nb.Nifti1Image(avg_data,img.affine,img.header)
    target_out_fn = fmap_fn.replace('.nii.gz','_avg.nii.gz')
    nb.save(new_img,target_out_fn)

In [ ]:
# now average these volumes to create most general template
for this_dir in [raw_target_dir,raw_dir]:
    data = []
    for fmap_fn in glob.glob(os.path.join(this_dir,'fmap','*avg.nii.gz')):
        img = nb.load(fmap_fn)
        data.append(img.get_data())
    avg_data = np.mean(data,axis=0)
    new_img = nb.Nifti1Image(avg_data,img.affine,img.header)
    base_out_fn = os.path.join(this_dir,'fmap','overall_avg.nii.gz')
    nb.save(new_img,base_out_fn)    

In [37]:
# now flirt avg fmap to target avg fmap
mat_fn = os.path.join(raw_dir,'fmap','overall_avg_to_target.mat')

flt = fsl.FLIRT()
flt.inputs.in_file = os.path.join(raw_dir,'fmap','overall_avg.nii.gz')
flt.inputs.reference = os.path.join(raw_target_dir,'fmap','overall_avg.nii.gz')
flt.inputs.out_matrix_file = mat_fn
flt.inputs.out_file = os.path.join(raw_dir,'fmap','overall_avg_flirted.nii.gz')
flt.run()

In [38]:
# now fnirt base to target
coef_fn = os.path.join(raw_dir,'fmap','overall_avg_field.nii.gz')

flt = fsl.FNIRT()
flt.inputs.in_file = os.path.join(raw_dir,'fmap','overall_avg.nii.gz')
flt.inputs.affine_file = mat_fn
flt.inputs.ref_file = os.path.join(raw_target_dir,'fmap','overall_avg.nii.gz')
flt.inputs.warped_file = os.path.join(raw_dir,'fmap','overall_avg_fnirted.nii.gz')
flt.inputs.field_file = coef_fn
flt.run()

In [40]:
# now apply warpfield to all niftis
for fn in glob.glob(os.path.join(raw_dir,'fmap','*epi.nii.gz')):
    flt = fsl.ApplyWarp()
    flt.inputs.ref_file = os.path.join(raw_target_dir,'fmap','overall_avg.nii.gz')
    flt.inputs.in_file = fn
    flt.inputs.out_file = fn.replace('.nii.gz','_fnirted.nii.gz')
    flt.inputs.field_file = coef_fn
#     flt.inputs.premat = mat_fn
    flt.run()
# applywarp --ref=MNI152_T1_2mm.nii --in=func.nii --out=funcInMNI.nii --warp=warp_struct2mni.nii --premat=func2struct.mat


In [42]:
# now apply warpfield to all niftis
for fn in glob.glob(os.path.join(raw_dir,'func','*bold.nii.gz')):
    flt = fsl.ApplyWarp()
    flt.inputs.ref_file = os.path.join(raw_target_dir,'fmap','overall_avg.nii.gz')
    flt.inputs.in_file = fn
    flt.inputs.out_file = fn.replace('.nii.gz','_fnirted.nii.gz')
    flt.inputs.field_file = coef_fn
#     flt.inputs.premat = mat_fn
    flt.run()
# applywarp --ref=MNI152_T1_2mm.nii --in=func.nii --out=funcInMNI.nii --warp=warp_struct2mni.nii --premat=func2struct.mat

